In [ ]:
import torch as tn
import torch.nn as nn
import torch.nn.functional as F
import torchtt as tntt
import numpy as np
import tt_iga
import matplotlib.pyplot as plt
import torch as tn
import datetime
import torch.utils.data 

tn.set_default_dtype(tn.float64)

In [ ]:

def create_geometry( ):
        
    Nt = 24                                                                
    lz = 40e-3                                                             
    Do = 72e-3                                                            
    Di = 51e-3                                                            
    hi = 13e-3                                                             
    bli = 3e-3                                                             
    Dc = 3.27640e-2                                                           
    # Dc = 32e-3
    # hc = 7e-3
    hc = 7.55176e-3                                                           
    ri = 20e-3                                                           
    ra = 18e-3                                                           
    blc = hi-hc                                                           
    rm = (Dc*Dc+hc*hc-ri*ri)/(Dc*np.sqrt(2)+hc*np.sqrt(2)-2*ri)                 
    R = rm-ri

    get_rO = lambda Ax,Ay,ri: (Ax**2+Ay**2-ri**2)/(np.sqrt(2)*(Ax+Ay)-2*ri)

    O = np.array([rm/np.sqrt(2),rm/np.sqrt(2)])
    alpha1 = -np.pi*3/4       
    alpha2 = np.math.asin((hc-rm/np.sqrt(2))/R)
    alpha = np.abs(alpha2-alpha1)
    
    A = np.array([[O[0] - ri/np.sqrt(2), O[1] - ri/np.sqrt(2)], [O[0] - Dc, O[1] - hc]])
    b = np.array([[A[0,0]*ri/np.sqrt(2)+A[0,1]*ri/np.sqrt(2)],[A[1,0]*Dc+A[1,1]*hc]])
    C = np.linalg.solve(A,b)

    def Cx(ri,Dc,hc):
        O = [((Dc*Dc+hc*hc-ri*ri)/(Dc*np.sqrt(2)+hc*np.sqrt(2)-2*ri) )/np.sqrt(2),((Dc*Dc+hc*hc-ri*ri)/(Dc*np.sqrt(2)+hc*np.sqrt(2)-2*ri) )/np.sqrt(2)]
        A = np.array([[O[0] - ri/np.sqrt(2), O[1] - ri/np.sqrt(2)], [O[0] - Dc, O[1] - hc]])
        b = np.array([[A[0,0]*ri/np.sqrt(2)+A[0,1]*ri/np.sqrt(2)],[A[1,0]*Dc+A[1,1]*hc]])
        C = np.linalg.solve(A,b)
        return C[0]

    def Cy(ri,Dc,hc):
        O = [((Dc*Dc+hc*hc-ri*ri)/(Dc*np.sqrt(2)+hc*np.sqrt(2)-2*ri) )/np.sqrt(2),( (Dc*Dc+hc*hc-ri*ri)/(Dc*np.sqrt(2)+hc*np.sqrt(2)-2*ri)  )/np.sqrt(2)]
        A = np.array([[O[0] - ri/np.sqrt(2), O[1] - ri/np.sqrt(2)], [O[0] - Dc, O[1] - hc]])
        b = np.array([[A[0,0]*ri/np.sqrt(2)+A[0,1]*ri/np.sqrt(2)],[A[1,0]*Dc+A[1,1]*hc]])
        C = np.linalg.solve(A,b)
        return C[1]

    control_points = tt_iga.geometry.ParameterDependentControlPoints([7,5])

    # params ri - 0 , Dc - 1 , Di - 2 
    control_points[:,0,0] = [0,0]
    control_points[:,1,0] = [lambda params: (Dc+params[1])/2, 0]
    control_points[:,2,0] = [lambda params: (Dc+params[1]), 0]
    control_points[:,3,0] = [(Dc+Di)/2,0]
    control_points[:,4,0] = [lambda params: Di+params[2],0]
    control_points[:,5,0] = [(Do+Di)/2,0]
    control_points[:,6,0] = [Do,0]
    
    control_points[:,0,1] = [lambda params: (ri+params[0])/np.sqrt(2)*0.5,lambda params: (ri+params[0])/np.sqrt(2)*0.5]
    control_points[:,1,1] = [lambda params: (Cx(ri,Dc,hc)+Dc/2)*0.5, lambda params: Cy(ri,Dc,hc)*0.5]
    control_points[:,2,1] = [lambda params: (Dc+params[1]), hc*0.5]
    control_points[:,3,1] = [(Dc+Di)/2,0.5*(hi-bli+hc)*0.5]
    control_points[:,4,1] = [lambda params: Di+params[2],(hi-bli)*0.5]
    control_points[:,5,1] = [(Do+Di)/2,(hi-bli)*0.5]
    control_points[:,6,1] = [Do,(hi-bli)*0.5]

    #control_points[:,0,1] = [lambda params: (ri+params[0])/np.sqrt(2)*0.5,lambda params: (ri+params[0])/np.sqrt(2)*0.5]
    #control_points[:,1,1] = [lambda params: (Cx(ri+params[0],Dc+params[1],hc)+(Dc+params[1])/2)*0.5, lambda params: Cy(ri+params[0],Dc+params[1],hc)*0.5]
    #control_points[:,2,1] = [lambda params: (Dc+params[1]), hc*0.5]
    #control_points[:,3,1] = [(Dc+Di)/2,0.5*(hi-bli+hc)*0.5]
    #control_points[:,4,1] = [lambda params: Di+params[2],(hi-bli)*0.5]
    #control_points[:,5,1] = [(Do+Di)/2,(hi-bli)*0.5]
    #control_points[:,6,1] = [Do,(hi-bli)*0.5]
    
    control_points[:,0,2] = [lambda params: (ri+params[0])/np.sqrt(2),lambda params: (ri+params[0])/np.sqrt(2)]
    control_points[:,1,2] = [lambda params: Cx(ri+params[0],Dc+params[1],hc), lambda params: Cy(ri+params[0],Dc+params[1],hc)]
    control_points[:,2,2] = [lambda params: (Dc+params[1]), hc]
    control_points[:,3,2] = [(Dc+Di)/2,0.5*(hi-bli+hc)]
    control_points[:,4,2] = [lambda params: Di+params[2],hi-bli]
    control_points[:,5,2] = [(Do+Di)/2,hi-bli]
    control_points[:,6,2] = [Do,hi-bli]
    
    control_points[:,0,3] = [(0.75*ri+0.25*Do)/np.sqrt(2),(0.75*ri+0.25*Do)/np.sqrt(2)]
    control_points[:,1,3] = [0.5*((0.75*ri+0.25*Do)/np.sqrt(2)+Dc+blc), 0.5*((0.75*ri+0.25*Do)/np.sqrt(2)+hi)]
    control_points[:,2,3] = [Dc+blc,lambda params:  hi+params[3]]
    control_points[:,3,3] = [ 0.5*(Dc+blc+Di-bli),lambda params:  hi+params[3]]
    control_points[:,4,3] = [ Di-bli, lambda params: hi+params[3]]
    control_points[:,5,3] = [lambda params: 0.5*(Di+Do),hi]
    control_points[:,6,3] = [Do,hi]

    control_points[:,0,4] = [Do/np.sqrt(2), Do/np.sqrt(2)]
    control_points[:,1,4] = [Do/np.sqrt(2)*(1-0.5*Dc/Do)+Do*(0.5*Dc/Do), Do/np.sqrt(2)*(1-0.5*Dc/Do)+Do*np.tan(np.pi/8)*(0.5*Dc/Do)]
    control_points[:,2,4] = [Do/np.sqrt(2)*(1-Dc/Do)+Do*(Dc/Do), Do/np.sqrt(2)*(1-Dc/Do)+Do*np.tan(np.pi/8)*(Dc/Do)]
    control_points[:,3,4] = [Do/np.sqrt(2)*(1-0.5*(Dc+Di)/Do)+Do*(0.5*(Dc+Di)/Do), Do/np.sqrt(2)*(1-0.5*(Dc+Di)/Do)+Do*np.tan(np.pi/8)*(0.5*(Dc+Di)/Do)]
    control_points[:,4,4] = [Do/np.sqrt(2)*(1-Di/Do)+Do*(Di/Do), Do/np.sqrt(2)*(1-Di/Do)+Do*np.tan(np.pi/8)*(Di/Do)]
    control_points[:,5,4] = [Do/np.sqrt(2)*(1-0.5*(Di+Do)/Do)+Do*(0.5*(Di+Do)/Do), Do/np.sqrt(2)*(1-0.5*(Di+Do)/Do)+Do*np.tan(np.pi/8)*(0.5*(Di+Do)/Do)]
    control_points[:,6,4] = [Do,Do*np.tan(np.pi/8)]

    weights = tt_iga.geometry.ParameterDependentWeights([7,5])
    weights[...] = 1.0
    alpha2 = lambda params: np.math.asin((hc-get_rO(Dc+params[1],hc,ri+params[0])/np.sqrt(2))/(get_rO(Dc+params[1],hc,ri+params[0])-ri-params[0]))
    weights[1,2] = lambda params :  np.sin((np.pi-np.abs(alpha2(params)+np.pi*3/4))/2)
    # weights[1,2] = lambda params :  np.abs(alpha2+np.pi*3/4)
    
    surface_excitation = lambda params: 0.5*(blc-params[...,1])*(hi+params[...,3]+hc)+(Di-bli-Dc-blc)*(hi+params[...,3])+0.5*(bli+params[...,2])*(hi+params[...,3]+hi-bli)
    return control_points, weights, surface_excitation

In [ ]:
control_points, weights, surface = create_geometry()

var1, var2, var3, var4 = (0.00075, 0.00075, 0.00075, 0.00075)

basis1 = tt_iga.bspline.BSplineBasis(np.array([0,0.4,0.4,0.6,0.6,1]),2)
basis2 = tt_iga.bspline.BSplineBasis(np.array([0,0.15,0.3,0.5,1]),1)
basis = [basis1, basis2]
basis_solution = [tt_iga.bspline.BSplineBasis(np.concatenate((np.linspace(0,0.4,21), np.linspace(0.4,0.6,21),np.linspace(0.6,1,21))),2)]
basis_solution.append(tt_iga.bspline.BSplineBasis(np.concatenate((np.linspace(0,0.15,16),np.linspace(0.15,0.3,15), np.linspace(0.3,0.5,15),np.linspace(0.5,1,17))),2))

nl = 10
basis_param = [tt_iga.lagrange.LagrangeLeg(nl,[-var1,var1]), tt_iga.lagrange.LagrangeLeg(nl,[-var2,var2]), tt_iga.lagrange.LagrangeLeg(nl,[-var3,var3]), tt_iga.lagrange.LagrangeLeg(nl,[-var4,var4])]
geom = tt_iga.PatchNURBS.interpolate_parameter_dependent(control_points, weights, basis, basis_param, eps = 1e-13)


In [ ]:
mu0 = 4*np.pi*1e-7
mur = 1500
mu_ref = lambda y: 1/mu0*((y[...,1]<0.5)*(y[...,0]<0.6)*(y[...,0]>0.4)+(y[...,1]<0.3)*(y[...,0]<0.4))+1/(mu0*mur)*tn.logical_not((y[...,1]<0.5)*(y[...,0]<0.6)*(y[...,0]>0.4)+(y[...,1]<0.3)*(y[...,0]<0.4))

tme = datetime.datetime.now()
Mass_tt = geom.mass_interp(basis_solution, eps = 1e-10)
tme = datetime.datetime.now() - tme
print('Time mass ', tme)
tme = datetime.datetime.now()
Stiff_tt = geom.stiffness_interp(basis_solution, eps = 1e-10, func_reference=mu_ref, qtt = True, verb = False, device = 'cuda:0')
tme = datetime.datetime.now() - tme
print('Time stiffness ', tme)

Jref = lambda y: 24*45/surface(y[...,2:])*(y[...,1]<0.5)*(y[...,0]<0.6)*(y[...,0]>0.4)+0.0
# Jref = lambda y: 5000000*(y[...,1]<0.5)*(y[...,0]<0.6)*(y[...,0]>0.4)+0.0

rhs_tt = geom.rhs_interp(basis_solution,Jref)

P1 = tn.eye(Mass_tt.N[0])
P2 = tn.eye(Mass_tt.N[1])
P2[-1,-1] = 0
P1[0,0] = 0
P1[-1,-1] = 0
Pin_tt = tntt.rank1TT([P1,P2]) ** tntt.eye([nl]*4)
Pbd_tt = (tntt.eye(Mass_tt.N) - Pin_tt) ** tntt.eye([nl]*4) / Mass_tt.N[0]

M_tt = (Pin_tt@Stiff_tt+Pbd_tt).round(1e-10)
rhs_tt = (Pin_tt @ rhs_tt + 0).round(1e-10)

print('System matrix... ',flush=True)

print('Rank Stiff', Stiff_tt.R, ' storage [MB] ',tntt.numel(Stiff_tt)*8/1e6)
print('Rank Mass', Mass_tt.R, ' storage [MB] ',tntt.numel(Mass_tt)*8/1e6)
print('Rank Mtt ',M_tt.R, ' storage [MB] ',tntt.numel(M_tt)*8/1e6)
print('Rank rhstt ',rhs_tt.R, ' storage [MB] ',tntt.numel(rhs_tt)*8/1e6)


In [ ]:
# tme = datetime.datetime.now() 
# dev_name = 'cuda:0'
# dofs_tt = tntt.solvers.amen_solve(M_tt.to(dev_name), rhs_tt.to(dev_name), x0 = tntt.ones(rhs_tt.N).to(dev_name), rmax = 300, eps = 1e-6, nswp = 50, kickrank = 8, preconditioner = 'c', verbose = False).cpu()
# tme = datetime.datetime.now() - tme
# print('Time system solve ',tme,flush=True)
# fspace = tt_iga.Function(basis_solution+basis_param)
# fspace.dofs = dofs_tt.round(1e-10)

In [ ]:
def generate_batch(n):
   # tn.manual_seed(123)
    inputs = tn.rand((n,4))*2-1
    outputs = []
    varz = [var1,var2,var3,var4]

    for p in inputs:
        outputs.append(M_tt[:,:,i1,i2,i3,i4,:,:,i1,i2,i3,i4])

    return inputs, outputs

n_train = 10000
train_ins, train_outs = generate_batch(n_train)
train_data = torch.utils.data.TensorDataset(train_ins, train_outs)

n_test = 1000
test_ins, test_outs = generate_batch(n_test)
test_data = torch.utils.data.TensorDataset(test_ins, test_outs)

dataloader_train = tn.utils.data.DataLoader(train_data, batch_size=100, shuffle=True, num_workers=10, pin_memory = False)
dataloader_test = tn.utils.data.DataLoader(test_data, batch_size=100, shuffle=True, num_workers=10, pin_memory = False)

In [ ]:
class CPINN(nn.Module):
   def __init__(self):
      super(CPINN, self).__init__()
      self.fc1 = nn.Linear(4, 50)
      self.fc2 = nn.Linear(50, 169)
      self.c1 = nn.ConvTranspose2d(1, 1, (6,6), stride=2)
      self.c2 = nn.ConvTranspose2d(1, 1, (6,6), stride=2)
   def forward(self, x):
      x = tn.tanh(self.fc1(x))
      x = tn.tanh(self.fc2(x))
      x = x.view([-1,1,13,13])
      x = tn.relu(self.c1(x))
      x = self.c2(x)
      return x.view([-1,64,64])

class ANN(nn.Module):
   def __init__(self):
      super(ANN, self).__init__()
      self.fc1 = nn.Linear(4, 25)
      self.fc2 = nn.Linear(25, 25)
      self.fc3 = nn.Linear(25, 64*64)
   def forward(self, x):
      x = tn.tanh(self.fc1(x))
      x = tn.tanh(self.fc2(x))
      x = (self.fc3(x))
      return x.view([-1,64,64])

class TTN(nn.Module):
    def __init__(self):
        super().__init__()
        p = 0.25
        self.fc1 = nn.Linear(4, 16)
        self.fc2 = nn.Linear(16, 16)
        self.ttl1 = tntt.nn.LinearLayerTT([2,2,2,2], [4,4,4,4], [1,4,4,4,1], initializer = 'He', dtype = tn.float64)
        self.dropout1 = nn.Dropout(p)
        self.ttl2 = tntt.nn.LinearLayerTT([4,4,4,4], [6,6,6,6], [1,4,4,4,1], initializer = 'He', dtype = tn.float64)
        self.dropout2 = nn.Dropout(p)
        self.ttl3 = tntt.nn.LinearLayerTT([6,6,6,6], [8,8,8,8], [1,4,4,4,1], initializer = 'He', dtype = tn.float64)
        self.dropout3 = nn.Dropout(p)
        self.ttl4 = tntt.nn.LinearLayerTT([8,8,8,8], [8,8,8,8], [1,2,2,2,1], initializer = 'He', dtype = tn.float64)

    def forward(self, x):
        x = tn.tanh(self.fc1(x))
        x = tn.tanh(self.fc2(x))
        x = x.view([-1,2,2,2,2])
        
        x = self.ttl1(x)
        #x = self.dropout1(x)
        x = tn.relu(x)

        x = self.ttl2(x)
        #x = self.dropout2(x)
        x = tn.relu(x)

        x = self.ttl3(x)
        #x = self.dropout3(x)
        x = tn.relu(x)

        x = self.ttl4(x)
        
        return x.view([-1,64,64])

In [ ]:
tn.cuda.empty_cache() 
device_name = 'cuda:1'
model = TTN()
print(model)
model.to(device_name) #.to(tn.float32)



loss_function = nn.MSELoss()


In [ ]:
# optimizer = tn.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
optimizer = tn.optim.Adam(model.parameters(), lr=0.005)
scheduler = tn.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)
device_name = 'cuda:0'
model.to(device_name).to(tn.float32)

tn.cuda.empty_cache() 

n_epochs = 400


history_test_loss = []
history_train_loss = []



for epoch in range(n_epochs):
    
    loss_train = 0

    model.train(True)
    for i,(param,sol) in enumerate(dataloader_train):
        
        param = param.to(device_name).to(tn.float32)
        sol = sol.to(device_name).to(tn.float32)
        
        optimizer.zero_grad()
        # Make predictions for this batch
        out = model(param)
        # Compute the loss and its gradients
        loss = (loss_function(out, sol))
        # regularization
        l2_lambda = 0.000005
        l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
        loss = loss+l2_lambda*l2_norm
        loss.backward()
        loss_train += loss

    loss_train /= len(dataloader_train)
    model.train(False)
    optimizer.step()

    loss_test = 0
    # for i,(param,sol) in enumerate(dataloader_train):
    #     
    #     param = param.to(device_name).to(tn.float32)
    #     sol = sol.to(device_name).to(tn.float32)
    #     
    #     
    #     # Make predictions for this batch
    #     out = model(param)
    #     # Compute the loss and its gradients
    #     loss = tn.sqrt(loss_function(out, sol))
    #     # regularization
    #     l2_lambda = 0.00005
    #     l2_norm = sum(p.pow(2.0).sum() for p in model.parameters())
    #     loss = loss+0#l2_lambda*l2_norm
    #     loss_test += loss

    loss_test /= len(dataloader_test)

    print('Epoch %d/%d: iterations %d, loss train %e, loss test %e'%(epoch+1, n_epochs, len(dataloader_train), loss_train, loss_test))
    history_train_loss.append(loss_train)
    history_test_loss.append(loss_test)

    tn.cuda.empty_cache() 
    
    

    
        

In [ ]:
i, o = generate_batch(2)

n = model(i.to(device_name).to(tn.float32)).cpu()

plt.figure()
plt.imshow(o[0,...])
plt.colorbar()

plt.figure()
plt.imshow(n[0,...].detach())
plt.colorbar()

plt.figure()
plt.imshow(n[0,...].detach()-o[0,...])
plt.colorbar()

In [ ]:
sum([ tn.numel(t) for t in list(model.parameters())])
dofs_tt